## Parsing Quantlet Directories

Objective: to parse Quantlet directories with code and metainfo files and create a dataframe for further analysis 

#### 1. Preparations

In [ ]:
##### INSTALL DEPENDENCIES AND SAVE SESSION
%pip install pandas==1.3.5
%pip install levenshtein
%pip freeze > session_info_sunflower.txt 

In [ ]:
##### IMPORT DEPENDENCIES

import os 
import pickle 
import sys

import importlib
sys.path.append('../3-data-preprocessing')

import preprocessing_utils # type: ignore
importlib.reload(preprocessing_utils)

import traverse_and_parse as tp  # noqa: E402
importlib.reload(tp)

##### CONSTANTS AND PATHS

# VARIABLE PARAMETERS
TODAY = '20231104'

# PATHS 
PATH_TO_PARSE = '../../data/QuantLet'
FILE_TYPES = ['m', 'py', 'r', 'R', 'M', 'ipynb']
TEST_PATH = '../data/Q_test'
OUTPUT_PATH = f"../../data/preprocessed/Quantlet/{TODAY}"

# CONSTANTS
IPY_CONVERT = False

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

2. Traversing and Parsing

In [ ]:
##### COLLECT, PARSE, AND CONCAT

output = tp.traverse_folder(PATH_TO_PARSE, FILE_TYPES)
repos = tp.prepare_repos_df(output['repos'])

repos.type_script = repos.type_script.str.lower()
repos = repos[repos.type_script.isin(
            ['py', 'r', 'm', 'ipynb']
        )
    ]
repos_df = repos[~repos.folder_name.str.contains('.ipynb_checkpoints')]
repos_df.loc[repos_df.type_script== 'ipynb', 'type_script']='py'
repos_df = repos_df.reset_index(drop=True)
print(repos_df.shape)

NAME = f'{OUTPUT_PATH}/Quantlets_{TODAY}'

##### SAVE THE DATAFRAME

with open(f'{NAME}.pkl', 'wb') as file:
    pickle.dump(repos_df, file)